# Module 06: GCP Vertex AI Advanced Features

**Difficulty**: ⭐⭐⭐
**Estimated Time**: 65 minutes
**Prerequisites**: 
- [Module 05: Google Cloud AI Platform Basics](05_google_cloud_ai_platform_basics.ipynb)
- Understanding of ML pipelines and deployment
- Familiarity with GCP services

## Learning Objectives

By the end of this notebook, you will be able to:
1. Use Vertex AI Workbench for unified ML development
2. Apply AutoML for Tables, Vision, and NLP tasks
3. Submit custom training jobs with distributed training
4. Build ML pipelines with Vertex AI Pipelines (Kubeflow)
5. Deploy models to Vertex AI endpoints with monitoring
6. Leverage Feature Store for feature management

## 1. Setup and Introduction

### Vertex AI Overview

Vertex AI is GCP's unified ML platform that consolidates all ML services.

```
┌─────────────────────────────────────────────────────────┐
│           Vertex AI Platform Architecture                │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  ┌───────────────────────────────────────────────────┐ │
│  │             Vertex AI Workbench                   │ │
│  │  - Managed Notebooks (JupyterLab)                 │ │
│  │  - User-managed Notebooks                         │ │
│  │  - Colab Enterprise Integration                   │ │
│  └───────────────────────────────────────────────────┘ │
│                          │                              │
│                          ▼                              │
│  ┌───────────────────────────────────────────────────┐ │
│  │              Data & Features                      │ │
│  │  ┌──────────────┐    ┌──────────────────┐        │ │
│  │  │ Datasets     │    │ Feature Store    │        │ │
│  │  │ - Managed    │    │ - Online serving │        │ │
│  │  │ - BigQuery   │    │ - Offline batch  │        │ │
│  │  └──────────────┘    └──────────────────┘        │ │
│  └───────────────────────────────────────────────────┘ │
│                          │                              │
│                          ▼                              │
│  ┌───────────────────────────────────────────────────┐ │
│  │               Training                            │ │
│  │  ┌──────────────┐    ┌──────────────────┐        │ │
│  │  │ AutoML       │    │ Custom Training  │        │ │
│  │  │ - Tables     │    │ - Distributed    │        │ │
│  │  │ - Vision     │    │ - Hyperparameter │        │ │
│  │  │ - NLP        │    │ - GPU/TPU        │        │ │
│  │  └──────────────┘    └──────────────────┘        │ │
│  └───────────────────────────────────────────────────┘ │
│                          │                              │
│                          ▼                              │
│  ┌───────────────────────────────────────────────────┐ │
│  │            Pipelines & Experiments                │ │
│  │  - Vertex AI Pipelines (Kubeflow)                 │ │
│  │  - Experiment tracking                            │ │
│  │  - Model registry                                 │ │
│  └───────────────────────────────────────────────────┘ │
│                          │                              │
│                          ▼                              │
│  ┌───────────────────────────────────────────────────┐ │
│  │              Deployment                           │ │
│  │  - Prediction endpoints                           │ │
│  │  - Batch prediction                               │ │
│  │  - Model monitoring                               │ │
│  │  - Explainability                                 │ │
│  └───────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────┘
```

**Benefits**:
- Unified platform (no switching between services)
- MLOps capabilities built-in
- Integration with BigQuery, Cloud Storage, etc.

In [ ]:
# Setup and imports
import numpy as np
import pandas as pd
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Mock Vertex AI SDK for demonstration
# In production, use: from google.cloud import aiplatform

class MockVertexAI:
    """Simulates Vertex AI client"""
    
    def __init__(self, project, location='us-central1'):
        self.project = project
        self.location = location
        self.initialized = True
    
    def init(self):
        """Initialize Vertex AI SDK"""
        print(f"✓ Vertex AI initialized")
        print(f"  Project: {self.project}")
        print(f"  Location: {self.location}")
        return self

## 2. Vertex AI Workbench

Managed notebook environment with integrated ML tools.

### Workbench Features

```
Vertex AI Workbench
├── Managed Notebooks
│   ├── Fully managed (no infrastructure)
│   ├── Auto-shutdown
│   ├── Git integration
│   └── Built-in Vertex AI access
│
├── User-Managed Notebooks
│   ├── More control over environment
│   ├── Custom docker images
│   ├── Root access
│   └── Persistent VMs
│
└── Integrated Tools
    ├── TensorBoard
    ├── What-If Tool
    ├── Data labeling
    └── Model deployment
```

In [ ]:
# Workbench instance configuration

def create_workbench_instance():
    """Configure Vertex AI Workbench instance"""
    config = {
        'name': 'ml-workbench-001',
        'type': 'managed',  # or 'user-managed'
        'machine_type': 'n1-standard-4',
        'accelerator': {
            'type': 'NVIDIA_TESLA_T4',
            'count': 1
        },
        'boot_disk': {
            'type': 'PD_SSD',
            'size_gb': 100
        },
        'data_disk': {
            'type': 'PD_STANDARD',
            'size_gb': 500
        },
        'runtime': {
            'python_version': '3.10',
            'frameworks': ['tensorflow', 'pytorch', 'sklearn']
        },
        'idle_shutdown_timeout': 60,  # minutes
        'labels': {
            'team': 'ml-team',
            'environment': 'development'
        }
    }
    
    print("💰 Workbench Cost Tips:")
    print("- Managed notebooks auto-shutdown when idle")
    print("- Use standard VMs (not highmem) initially")
    print("- Add GPU only when training large models")
    print("- Set aggressive idle shutdown timeout\n")
    
    return config

workbench = create_workbench_instance()
print("Vertex AI Workbench Configuration:")
print(json.dumps(workbench, indent=2))

## 3. AutoML on Vertex AI

Automated ML for common tasks without writing training code.

### AutoML Task Types

```
┌─────────────────────────────────────────────────────────┐
│           Vertex AI AutoML Options                       │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  AutoML Tables (Tabular Data)                           │
│  ├─ Classification                                      │
│  ├─ Regression                                          │
│  ├─ Forecasting                                         │
│  └─ Auto feature engineering                            │
│                                                         │
│  AutoML Vision                                          │
│  ├─ Image classification                                │
│  ├─ Object detection                                    │
│  ├─ Image segmentation                                  │
│  └─ Transfer learning                                   │
│                                                         │
│  AutoML Natural Language                                │
│  ├─ Text classification                                 │
│  ├─ Entity extraction                                   │
│  ├─ Sentiment analysis                                  │
│  └─ Custom models                                       │
│                                                         │
│  AutoML Video                                           │
│  ├─ Video classification                                │
│  ├─ Object tracking                                     │
│  └─ Action recognition                                  │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# AutoML Tables configuration

def create_automl_tabular_training():
    """Configure AutoML for tabular data"""
    training_config = {
        'display_name': 'churn_prediction_automl',
        'dataset': 'projects/my-project/locations/us-central1/datasets/123456',
        'target_column': 'churn',
        'prediction_type': 'classification',
        'optimization_objective': 'maximize-au-roc',
        'transformations': {
            'auto': True,  # Automatic feature engineering
            'column_transformations': [
                {'column': 'age', 'transformation': 'numeric'},
                {'column': 'tenure', 'transformation': 'numeric'},
                {'column': 'contract_type', 'transformation': 'categorical'},
                {'column': 'monthly_charges', 'transformation': 'numeric'}
            ]
        },
        'budget_milli_node_hours': 1000,  # ~1 node hour = $3
        'split': {
            'training_fraction': 0.8,
            'validation_fraction': 0.1,
            'test_fraction': 0.1
        },
        'export_evaluated_data_items': True
    }
    
    print("AutoML Tables Training:")
    print(f"Budget: {training_config['budget_milli_node_hours']/1000} node-hours")
    print(f"Estimated cost: ~${training_config['budget_milli_node_hours']/1000 * 3:.2f}")
    print("\n⏱️  Training typically takes 2-6 hours\n")
    
    return training_config

automl_config = create_automl_tabular_training()
print(json.dumps(automl_config, indent=2))

In [ ]:
# Simulate AutoML training

class AutoMLTrainer:
    """Simulates Vertex AI AutoML training"""
    
    def __init__(self, display_name, objective):
        self.display_name = display_name
        self.objective = objective
        self.state = 'PENDING'
        self.models_tried = []
    
    def train(self, max_trials=20):
        """Run AutoML training with multiple trials"""
        print(f"Starting AutoML training: {self.display_name}")
        print(f"Objective: {self.objective}\n")
        
        self.state = 'RUNNING'
        
        algorithms = [
            'Gradient Boosted Trees',
            'Random Forest',
            'Neural Network',
            'Linear Model',
            'Ensemble'
        ]
        
        for i in range(max_trials):
            algo = np.random.choice(algorithms)
            score = 0.65 + np.random.rand() * 0.3
            
            model_trial = {
                'trial': i + 1,
                'algorithm': algo,
                'score': score
            }
            self.models_tried.append(model_trial)
            
            if (i + 1) % 5 == 0:
                print(f"Trial {i+1}/{max_trials}: {algo} - Score: {score:.4f}")
        
        # Find best model
        best_model = max(self.models_tried, key=lambda x: x['score'])
        self.state = 'SUCCEEDED'
        
        print(f"\n✓ Training completed!")
        print(f"Best model: {best_model['algorithm']}")
        print(f"Best score: {best_model['score']:.4f}")
        
        return best_model

In [ ]:
# Run AutoML simulation
automl = AutoMLTrainer(
    display_name='churn_prediction_automl',
    objective='maximize-au-roc'
)

best_model = automl.train(max_trials=20)

print("\nAutoML Benefits:")
print("- No manual feature engineering")
print("- Tries multiple algorithms automatically")
print("- Handles data preprocessing")
print("- Provides model explanations")
print("- Fast deployment to endpoints")

## 4. Custom Training Jobs

Run custom training code with full control over the training process.

### Custom Training Architecture

```
┌─────────────────────────────────────────────────────────┐
│          Custom Training on Vertex AI                    │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Training Code                                          │
│  ┌──────────────────────────────┐                      │
│  │ Python package or script     │                      │
│  │ - train.py (entry point)     │                      │
│  │ - model code                 │                      │
│  │ - requirements.txt           │                      │
│  └──────────────────────────────┘                      │
│           │                                             │
│           ▼                                             │
│  Container Image                                        │
│  ┌──────────────────────────────┐                      │
│  │ Pre-built:                   │                      │
│  │ - TensorFlow                 │                      │
│  │ - PyTorch                    │                      │
│  │ - Scikit-learn               │                      │
│  │ Custom:                      │                      │
│  │ - Your own Dockerfile        │                      │
│  └──────────────────────────────┘                      │
│           │                                             │
│           ▼                                             │
│  Training Job                                           │
│  ┌──────────────────────────────┐                      │
│  │ Compute:                     │                      │
│  │ - Single VM                  │                      │
│  │ - Distributed (multi-VM)     │                      │
│  │ - GPU/TPU support            │                      │
│  │                              │                      │
│  │ Features:                    │                      │
│  │ - Hyperparameter tuning      │                      │
│  │ - Checkpointing              │                      │
│  │ - TensorBoard integration    │                      │
│  └──────────────────────────────┘                      │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Custom training job configuration

def create_custom_training_job():
    """Configure custom training job on Vertex AI"""
    job_config = {
        'display_name': 'custom-classifier-training',
        'container_spec': {
            'image_uri': 'gcr.io/cloud-aiplatform/training/tf-cpu.2-11:latest',
            'command': ['python', 'task.py'],
            'args': [
                '--train-data', 'gs://my-bucket/data/train.csv',
                '--eval-data', 'gs://my-bucket/data/eval.csv',
                '--model-dir', 'gs://my-bucket/models/output',
                '--epochs', '20',
                '--batch-size', '32'
            ]
        },
        'machine_spec': {
            'machine_type': 'n1-standard-8',
            'accelerator_type': 'NVIDIA_TESLA_T4',
            'accelerator_count': 1
        },
        'replica_count': 1,  # Single-node training
        'scheduling': {
            'timeout': '7200s',  # 2 hours
            'restart_job_on_worker_restart': True
        },
        'service_account': 'vertex-ai-sa@my-project.iam.gserviceaccount.com',
        'tensorboard': 'projects/my-project/locations/us-central1/tensorboards/123'
    }
    return job_config

# Distributed training configuration
def create_distributed_training_job():
    """Configure distributed training (multi-node)"""
    job_config = {
        'display_name': 'distributed-training',
        'worker_pool_specs': [
            {  # Chief worker
                'machine_spec': {
                    'machine_type': 'n1-standard-16',
                    'accelerator_type': 'NVIDIA_TESLA_V100',
                    'accelerator_count': 2
                },
                'replica_count': 1,
                'container_spec': {
                    'image_uri': 'gcr.io/my-project/trainer:latest',
                    'command': ['python', '-m', 'trainer.task']
                }
            },
            {  # Worker pool
                'machine_spec': {
                    'machine_type': 'n1-standard-16',
                    'accelerator_type': 'NVIDIA_TESLA_V100',
                    'accelerator_count': 2
                },
                'replica_count': 3,  # 3 worker nodes
                'container_spec': {
                    'image_uri': 'gcr.io/my-project/trainer:latest',
                    'command': ['python', '-m', 'trainer.task']
                }
            }
        ]
    }
    return job_config

single_node = create_custom_training_job()
distributed = create_distributed_training_job()

print("Single-Node Training:")
print(json.dumps(single_node, indent=2))
print("\nDistributed Training:")
print(json.dumps(distributed, indent=2))

## 5. Vertex AI Pipelines

Build reproducible ML workflows using Kubeflow Pipelines.

### Pipeline Architecture

```
┌─────────────────────────────────────────────────────────┐
│         Vertex AI Pipelines (Kubeflow)                   │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Pipeline = DAG of Components                           │
│                                                         │
│         ┌──────────────┐                                │
│         │ Data Ingestion│                               │
│         └──────┬────────┘                               │
│                │                                        │
│                ▼                                        │
│         ┌──────────────┐                                │
│         │ Preprocessing │                               │
│         └──────┬────────┘                               │
│                │                                        │
│       ┌────────┴────────┐                               │
│       │                 │                               │
│       ▼                 ▼                               │
│  ┌─────────┐      ┌─────────┐                          │
│  │ Train   │      │ Train   │  (Parallel)              │
│  │ Model A │      │ Model B │                          │
│  └────┬────┘      └────┬────┘                          │
│       │                │                                │
│       └────────┬────────┘                               │
│                ▼                                        │
│         ┌──────────────┐                                │
│         │  Evaluation  │                                │
│         └──────┬────────┘                               │
│                │                                        │
│                ▼                                        │
│         ┌──────────────┐                                │
│         │ Conditional  │                                │
│         │ Deployment   │  (if metrics > threshold)     │
│         └──────────────┘                                │
│                                                         │
│  Features:                                              │
│  - Reusable components                                  │
│  - Caching for faster runs                              │
│  - Experiment tracking                                  │
│  - Version control                                      │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Define pipeline components (conceptual)

def create_pipeline_definition():
    """
    Define ML pipeline structure
    In practice, use Kubeflow Pipelines SDK
    """
    pipeline_def = {
        'name': 'training_pipeline',
        'description': 'End-to-end ML training pipeline',
        'parameters': {
            'project_id': {'type': 'String'},
            'data_path': {'type': 'String'},
            'model_threshold': {'type': 'Float', 'default': 0.85}
        },
        'components': [
            {
                'name': 'data_ingestion',
                'type': 'ContainerOp',
                'image': 'gcr.io/my-project/data-ingestion:v1',
                'inputs': ['data_path'],
                'outputs': ['raw_data']
            },
            {
                'name': 'preprocessing',
                'type': 'ContainerOp',
                'image': 'gcr.io/my-project/preprocessing:v1',
                'inputs': ['raw_data'],
                'outputs': ['train_data', 'eval_data']
            },
            {
                'name': 'training',
                'type': 'CustomTrainingJobOp',
                'inputs': ['train_data', 'eval_data'],
                'outputs': ['model', 'metrics']
            },
            {
                'name': 'evaluation',
                'type': 'ContainerOp',
                'image': 'gcr.io/my-project/evaluation:v1',
                'inputs': ['model', 'eval_data'],
                'outputs': ['eval_metrics']
            },
            {
                'name': 'deploy_model',
                'type': 'ModelDeployOp',
                'condition': 'eval_metrics.auc >= model_threshold',
                'inputs': ['model'],
                'outputs': ['endpoint']
            }
        ],
        'execution': {
            'caching': True,
            'schedule': '0 2 * * *',  # Daily at 2 AM
        }
    }
    return pipeline_def

pipeline = create_pipeline_definition()
print("Pipeline Definition:")
print(json.dumps(pipeline, indent=2))

In [ ]:
# Simulate pipeline execution

class VertexAIPipeline:
    """Simulates Vertex AI Pipeline execution"""
    
    def __init__(self, pipeline_name):
        self.pipeline_name = pipeline_name
        self.runs = []
    
    def run(self, parameters):
        """Execute pipeline with given parameters"""
        run_id = f"run_{len(self.runs) + 1}"
        
        print(f"Starting pipeline run: {run_id}")
        print(f"Parameters: {parameters}\n")
        
        # Simulate component execution
        components = [
            'data_ingestion',
            'preprocessing',
            'training',
            'evaluation',
            'deploy_model'
        ]
        
        run_info = {
            'run_id': run_id,
            'parameters': parameters,
            'component_results': {}
        }
        
        for component in components:
            print(f"Executing: {component}...")
            
            # Simulate component execution
            if component == 'evaluation':
                auc = 0.7 + np.random.rand() * 0.25
                run_info['component_results'][component] = {'auc': auc}
                print(f"  Metrics: AUC={auc:.4f}")
                
                # Conditional deployment
                if auc < parameters.get('model_threshold', 0.85):
                    print(f"  ⚠️  AUC below threshold, skipping deployment\n")
                    break
            elif component == 'deploy_model':
                print(f"  ✓ Model deployed to endpoint\n")
        
        run_info['status'] = 'SUCCEEDED'
        self.runs.append(run_info)
        
        print(f"Pipeline run {run_id} completed!")
        return run_info

In [ ]:
# Execute pipeline
pipeline_executor = VertexAIPipeline('training_pipeline')

run_result = pipeline_executor.run({
    'project_id': 'my-ml-project',
    'data_path': 'gs://my-bucket/data/train.csv',
    'model_threshold': 0.85
})

print("\nPipeline Benefits:")
print("- Reproducible workflows")
print("- Component reusability")
print("- Automated orchestration")
print("- Experiment tracking")
print("- Easy scheduling")

## 6. Model Deployment on Vertex AI

Deploy models to managed endpoints for predictions.

### Deployment Options

```
1. Online Prediction (real-time)
2. Batch Prediction (large datasets)
3. Private Endpoints (VPC)
```

In [ ]:
# Model deployment configuration

def deploy_model_to_endpoint():
    """Configure model deployment to Vertex AI endpoint"""
    deployment_config = {
        'endpoint': {
            'display_name': 'classifier-endpoint-prod',
            'description': 'Production classification endpoint',
            'labels': {
                'environment': 'production',
                'model': 'classifier'
            }
        },
        'deployed_model': {
            'display_name': 'classifier-v1',
            'model': 'projects/my-project/locations/us-central1/models/12345',
            'machine_type': 'n1-standard-4',
            'min_replica_count': 1,
            'max_replica_count': 5,
            'autoscaling': {
                'target_cpu_utilization': 60,
                'target_accelerator_duty_cycle': 60
            },
            'enable_access_logging': True,
            'enable_container_logging': True
        },
        'traffic_split': {
            'classifier-v1': 100  # 100% traffic to this version
        }
    }
    
    print("💰 Deployment Cost Optimization:")
    print("- Min replica count = 1 (scales down when idle)")
    print("- Use standard VMs (not highmem)")
    print("- Enable auto-scaling based on load")
    print("- Monitor and adjust replicas based on traffic\n")
    
    return deployment_config

deployment = deploy_model_to_endpoint()
print("Deployment Configuration:")
print(json.dumps(deployment, indent=2))

In [ ]:
# Simulate predictions

class VertexAIEndpoint:
    """Simulates Vertex AI prediction endpoint"""
    
    def __init__(self, endpoint_name):
        self.endpoint_name = endpoint_name
        self.prediction_count = 0
    
    def predict(self, instances):
        """
        Make predictions on instances
        
        Args:
            instances: List of input instances
        
        Returns:
            predictions with confidence scores
        """
        predictions = []
        
        for instance in instances:
            # Simulated prediction
            pred_class = np.random.choice([0, 1], p=[0.6, 0.4])
            confidence = np.random.rand()
            
            predictions.append({
                'class': int(pred_class),
                'confidence': float(confidence),
                'probabilities': {
                    '0': float(1 - confidence) if pred_class == 1 else float(confidence),
                    '1': float(confidence) if pred_class == 1 else float(1 - confidence)
                }
            })
        
        self.prediction_count += len(instances)
        return predictions
    
    def batch_predict(self, gcs_source, gcs_destination):
        """Run batch prediction on large dataset"""
        print(f"Batch prediction job started")
        print(f"Source: {gcs_source}")
        print(f"Destination: {gcs_destination}")
        
        # Simulate processing
        num_instances = np.random.randint(10000, 100000)
        print(f"\nProcessing {num_instances:,} instances...")
        print(f"✓ Batch prediction completed")
        print(f"Results saved to: {gcs_destination}")
        
        return {'instances_processed': num_instances}

In [ ]:
# Test endpoint predictions
endpoint = VertexAIEndpoint('classifier-endpoint-prod')

# Online predictions
print("Online Predictions:\n")
test_instances = [
    {'age': 45, 'tenure': 24, 'monthly_charges': 75.5},
    {'age': 32, 'tenure': 12, 'monthly_charges': 45.0}
]

predictions = endpoint.predict(test_instances)
for i, pred in enumerate(predictions):
    print(f"Instance {i+1}:")
    print(f"  Predicted class: {pred['class']}")
    print(f"  Confidence: {pred['confidence']:.2%}")
    print()

# Batch predictions
print("="*50)
print("\nBatch Predictions:\n")
batch_result = endpoint.batch_predict(
    gcs_source='gs://my-bucket/batch-input/*.csv',
    gcs_destination='gs://my-bucket/batch-predictions/'
)

## 7. Feature Store

Centralized repository for feature management and serving.

### Feature Store Architecture

```
┌─────────────────────────────────────────────────────────┐
│           Vertex AI Feature Store                        │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Feature Registry                                       │
│  ┌─────────────────────────────────────┐               │
│  │ Entity Types:                       │               │
│  │ - customer                          │               │
│  │ - product                           │               │
│  │                                     │               │
│  │ Features:                           │               │
│  │ - customer.age                      │               │
│  │ - customer.lifetime_value           │               │
│  │ - product.avg_rating                │               │
│  └─────────────────────────────────────┘               │
│                │                                        │
│                ▼                                        │
│  Dual Serving                                           │
│  ┌──────────────────┐   ┌──────────────────┐          │
│  │ Online Serving   │   │ Offline (Batch)  │          │
│  │ - Low latency    │   │ - Training data  │          │
│  │ - Real-time      │   │ - BigQuery       │          │
│  │ - Point lookup   │   │ - Historical     │          │
│  └──────────────────┘   └──────────────────┘          │
│                                                         │
│  Benefits:                                              │
│  - Feature reuse across models                         │
│  - Consistent online/offline features                  │
│  - Point-in-time correctness                           │
│  - Feature versioning                                  │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Feature Store configuration

def create_feature_store():
    """Configure Vertex AI Feature Store"""
    featurestore_config = {
        'name': 'customer_featurestore',
        'online_serving_config': {
            'fixed_node_count': 1  # Minimum for low traffic
        },
        'entity_types': [
            {
                'entity_type_id': 'customer',
                'description': 'Customer entity',
                'features': [
                    {
                        'feature_id': 'age',
                        'value_type': 'INT64',
                        'description': 'Customer age'
                    },
                    {
                        'feature_id': 'lifetime_value',
                        'value_type': 'DOUBLE',
                        'description': 'Customer lifetime value'
                    },
                    {
                        'feature_id': 'tenure_months',
                        'value_type': 'INT64',
                        'description': 'Months as customer'
                    },
                    {
                        'feature_id': 'avg_monthly_spend',
                        'value_type': 'DOUBLE',
                        'description': 'Average monthly spend'
                    }
                ]
            }
        ]
    }
    return featurestore_config

featurestore = create_feature_store()
print("Feature Store Configuration:")
print(json.dumps(featurestore, indent=2))

In [ ]:
# Simulate feature serving

class FeatureStore:
    """Simulates Vertex AI Feature Store"""
    
    def __init__(self, name):
        self.name = name
        self.features = {}
    
    def ingest_features(self, entity_type, entity_id, feature_values):
        """Ingest features for an entity"""
        key = f"{entity_type}:{entity_id}"
        self.features[key] = {
            'entity_type': entity_type,
            'entity_id': entity_id,
            'features': feature_values,
            'timestamp': datetime.now()
        }
        print(f"✓ Ingested features for {entity_type}:{entity_id}")
    
    def read_online(self, entity_type, entity_ids, feature_names):
        """
        Read features for online serving (real-time)
        Low latency point lookup
        """
        results = []
        
        for entity_id in entity_ids:
            key = f"{entity_type}:{entity_id}"
            if key in self.features:
                feature_data = self.features[key]
                result = {
                    'entity_id': entity_id,
                    'features': {
                        name: feature_data['features'].get(name)
                        for name in feature_names
                    }
                }
                results.append(result)
        
        return results
    
    def batch_read_offline(self, entity_type, feature_names, start_time, end_time):
        """
        Read features for offline training (batch)
        Point-in-time correctness
        """
        print(f"Reading features for training data...")
        print(f"Entity type: {entity_type}")
        print(f"Features: {feature_names}")
        print(f"Time range: {start_time} to {end_time}")
        
        # Simulate creating training dataset
        num_rows = np.random.randint(1000, 10000)
        print(f"\n✓ Generated {num_rows:,} training examples")
        
        return f"bq://my-project.dataset.training_features_{datetime.now().strftime('%Y%m%d')}"

In [ ]:
# Test Feature Store
fs = FeatureStore('customer_featurestore')

# Ingest features
print("Ingesting features:\n")
fs.ingest_features('customer', 'C001', {
    'age': 45,
    'lifetime_value': 5000.0,
    'tenure_months': 24,
    'avg_monthly_spend': 75.5
})

fs.ingest_features('customer', 'C002', {
    'age': 32,
    'lifetime_value': 2500.0,
    'tenure_months': 12,
    'avg_monthly_spend': 45.0
})

# Online serving (real-time prediction)
print("\n" + "="*50)
print("\nOnline feature serving (for real-time predictions):\n")
online_features = fs.read_online(
    entity_type='customer',
    entity_ids=['C001', 'C002'],
    feature_names=['age', 'tenure_months', 'avg_monthly_spend']
)

for result in online_features:
    print(f"Customer {result['entity_id']}: {result['features']}")

# Offline serving (training data)
print("\n" + "="*50)
print("\nOffline feature serving (for training):\n")
training_data = fs.batch_read_offline(
    entity_type='customer',
    feature_names=['age', 'lifetime_value', 'tenure_months', 'avg_monthly_spend'],
    start_time=datetime(2024, 1, 1),
    end_time=datetime(2024, 12, 31)
)
print(f"Training data location: {training_data}")

## Exercises

### Exercise 1: Pipeline Design

Design a complete Vertex AI Pipeline for fraud detection:
1. Ingest transaction data from BigQuery
2. Read features from Feature Store
3. Train model with hyperparameter tuning
4. Evaluate on holdout set
5. Deploy if precision > 0.90
6. Update Feature Store with predictions

Define all components and dependencies.

In [ ]:
# Your solution here
def design_fraud_detection_pipeline():
    """
    Design end-to-end fraud detection pipeline on Vertex AI
    
    Include:
    - Component definitions
    - Feature Store integration
    - Conditional deployment
    - Feedback loop
    
    Returns:
        dict with complete pipeline specification
    """
    # TODO: Design pipeline
    pass

# Test your pipeline
# pipeline = design_fraud_detection_pipeline()
# print(json.dumps(pipeline, indent=2))

### Exercise 2: AutoML vs Custom Training

Compare AutoML Tables vs Custom Training for a classification task:
- Dataset: 100K rows, 20 features
- Budget: $200
- Timeline: 1 week

Analyze trade-offs and recommend an approach.

In [ ]:
# Your solution here
def compare_training_approaches(dataset_size, budget, timeline_days):
    """
    Compare AutoML vs Custom Training
    
    Consider:
    - Cost (AutoML vs compute hours)
    - Time to production
    - Model performance
    - Customization needs
    - Maintenance
    
    Returns:
        dict with comparison and recommendation
    """
    # TODO: Analyze trade-offs
    pass

# Test your analysis
# comparison = compare_training_approaches(
#     dataset_size=100000,
#     budget=200,
#     timeline_days=7
# )
# print(json.dumps(comparison, indent=2))

### Exercise 3: Feature Store Strategy

Design a Feature Store for an e-commerce recommendation system:
- User features (demographics, behavior)
- Product features (category, price, ratings)
- Interaction features (clicks, purchases)

Define entity types, features, and serving strategy.

In [ ]:
# Your solution here
def design_recommendation_feature_store():
    """
    Design Feature Store for recommendations
    
    Include:
    - Entity types and features
    - Online vs offline serving
    - Update frequency
    - Point-in-time joins
    
    Returns:
        dict with Feature Store design
    """
    # TODO: Design Feature Store
    pass

# Test your design
# fs_design = design_recommendation_feature_store()
# print(json.dumps(fs_design, indent=2))

## Summary

In this notebook, you learned:

1. **Vertex AI Workbench**: Unified development environment
2. **AutoML**: Automated ML for Tables, Vision, and NLP
3. **Custom Training**: Distributed training with full control
4. **Vertex AI Pipelines**: Kubeflow-based ML workflows
5. **Model Deployment**: Online and batch prediction endpoints
6. **Feature Store**: Centralized feature management and serving

### Key Takeaways

- Vertex AI unifies all ML services in one platform
- Use AutoML for quick POCs, custom training for production
- Pipelines enable reproducible, automated workflows
- Feature Store ensures consistency between training and serving
- Always enable auto-scaling and monitoring for endpoints
- Start with managed services, customize only when needed

### Platform Comparison

| Feature | AWS SageMaker | Azure ML | GCP Vertex AI |
|---------|--------------|----------|---------------|
| Unified Platform | Separate services | Azure ML Studio | **Vertex AI** |
| AutoML | Autopilot | AutoML | **Tables, Vision, NLP** |
| Feature Store | ✓ | ✓ (preview) | **✓ (integrated)** |
| Pipelines | Step Functions | Pipelines | **Kubeflow** |
| BigQuery Integration | - | - | **✓ Native** |

### What's Next?

- Practice building end-to-end ML pipelines
- Explore model monitoring and explainability
- Experiment with distributed training
- Implement MLOps best practices

### Additional Resources

- [Vertex AI Documentation](https://cloud.google.com/vertex-ai/docs)
- [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines)
- [Feature Store Guide](https://cloud.google.com/vertex-ai/docs/featurestore)
- [AutoML Tutorial](https://cloud.google.com/vertex-ai/docs/tutorials/tabular-automl)

---

**Congratulations!** You've completed the Cloud ML Platforms module. You now understand how to leverage AWS SageMaker, Azure ML, and GCP Vertex AI for production ML deployments.